In [4]:
from keras.applications import MobileNet
from keras.layers import Dense 
from keras.layers import Dropout
from keras.layers import Flatten 
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D
from keras.layers import Conv2D


In [5]:
#Loading and removing top layer
img_rows,img_cols = 224,224

mobilenet = MobileNet(weights='imagenet',
                        include_top = False,
                        input_shape=(img_rows,img_cols,3)
                        )

W0613 02:58:15.867674 4628649408 deprecation_wrapper.py:119] From /Users/abuzaid/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0613 02:58:15.887339 4628649408 deprecation_wrapper.py:119] From /Users/abuzaid/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0613 02:58:15.894406 4628649408 deprecation_wrapper.py:119] From /Users/abuzaid/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0613 02:58:15.927129 4628649408 deprecation_wrapper.py:119] From /Users/abuzaid/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instea

In [14]:
#Freezing lower layer
for layer in mobilenet.layers:
    layer.trainable = False
    
for (i,layer) in enumerate(mobilenet.layers):
    print(str(i)+" "+ layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 Conv2D False
11 BatchNormalization False
12 ReLU False
13 ZeroPadding2D False
14 DepthwiseConv2D False
15 BatchNormalization False
16 ReLU False
17 Conv2D False
18 BatchNormalization False
19 Conv2D False
20 BatchNormalization False
21 ReLU False
22 DepthwiseConv2D False
23 BatchNormalization False
24 ReLU False
25 Conv2D False
26 BatchNormalization False
27 Add False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 ZeroPadding2D False
32 DepthwiseConv2D False
33 BatchNormalization False
34 ReLU False
35 Conv2D False
36 BatchNormalization False
37 Conv2D False
38 BatchNormalization False
39 ReLU False
40 DepthwiseConv2D False
41 BatchNormalization False
42 ReLU False
43 Conv2D False
44 BatchNormalization False
45 Add False
46 Conv2D False
47 BatchNormalization False
48 ReL

In [63]:
#Adding Top MOdel 
def addTopModelMobileNet(bottom_model,num_classes):
    
    top_model= bottom_model.output
    top_model =GlobalAveragePooling2D()(top_model)
    top_model=Dense(1024,activation='relu')(top_model)
    top_model=Dense(1024,activation='relu')(top_model)
    top_model=Dense(512,activation='relu')(top_model)
    top_model=Dense(num_classes,activation='softmax')(top_model)
    return top_model



In [64]:
#adding Fullyconnected Layer 

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 10

FC_Head = addTopModelMobileNet(mobilenet,num_classes= num_classes)

model = Model(inputs = mobilenet.input, outputs = FC_Head)

print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [30]:
import keras
print(keras.__version__)

2.2.4


In [65]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = './train'
validation_data_dir = './validation'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 1097 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [72]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping





# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 1097
nb_validation_samples = 272

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
68/68 [==============================] - 775s 11s/step - loss: 0.7708 - acc: 0.8101 - val_loss: 3.1178 - val_acc: 0.5820
Epoch 2/5
68/68 [==============================] - 760s 11s/step - loss: 0.5561 - acc: 0.8708 - val_loss: 4.3565 - val_acc: 0.5312
Epoch 3/5
68/68 [==============================] - 755s 11s/step - loss: 0.5477 - acc: 0.8671 - val_loss: 6.3407 - val_acc: 0.3359
Epoch 4/5
68/68 [==============================] - 814s 12s/step - loss: 0.4766 - acc: 0.8823 - val_loss: 9.9101 - val_acc: 0.2576
Epoch 5/5
68/68 [==============================] - 828s 12s/step - loss: 0.4485 - acc: 0.8970 - val_loss: 13.6971 - val_acc: 0.1406


In [11]:
from keras.models import load_model

classifier = load_model('monkey_breed_mobileNet.h5')                            







W0613 03:03:09.785529 4628649408 deprecation_wrapper.py:119] From /Users/abuzaid/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0613 03:03:09.911311 4628649408 deprecation.py:323] From /Users/abuzaid/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

monkey_breeds_dict = {"[0]": "mantled_howler ", 
                      "[1]": "patas_monkey",
                      "[2]": "bald_uakari",
                      "[3]": "japanese_macaque",
                      "[4]": "pygmy_marmoset ",
                      "[5]": "white_headed_capuchin",
                      "[6]": "silvery_marmoset",
                      "[7]": "common_squirrel_monkey",
                      "[8]": "black_headed_night_monkey",
                      "[9]": "nilgiri_langur"}

monkey_breeds_dict_n = {"n0": "mantled_howler ", 
                      "n1": "patas_monkey",
                      "n2": "bald_uakari",
                      "n3": "japanese_macaque",
                      "n4": "pygmy_marmoset ",
                      "n5": "white_headed_capuchin",
                      "n6": "silvery_marmoset",
                      "n7": "common_squirrel_monkey",
                      "n8": "black_headed_night_monkey",
                      "n9": "nilgiri_langur"}

def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("./validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - pygmy_marmoset 
Class - white_headed_capuchin
Class - mantled_howler 
Class - white_headed_capuchin
Class - pygmy_marmoset 
Class - patas_monkey
Class - japanese_macaque
Class - mantled_howler 
Class - white_headed_capuchin
Class - common_squirrel_monkey
